In [1]:
from src.all_in_one import *

No password for tt_pt_FM_v04.11.xlsx


In [2]:
cds = pd.read_csv(r"notebooks/consolidate_datasets/dec24/consolidated_dataset_ptfu.csv")

In [3]:
# Inventing new two rid as a placeholder for the dataset be able to merged with beneficiaries with no rid.
cds.loc[cds.index.max() + 1, "rid"] = "M0001"
cds.loc[cds.index.max() + 1, "rid"] = "M0002"

In [ ]:
# TD Dataset
td = pd.read_excel(
    r"data/raw/1224/other_services/TD Tracking Tools - December 2024.xlsx"
)
td = td[
    [
        "TDSP",
        "Ref #",
        "FC ID #",
        "CSLT",
        "CSNT",
        "LD",
        "IU",
        "CW",
        "Com",
        "Adv",
        "LD ADMSP",
        "LD SJAC",
        "IS#1",
        "IS#2",
        "IS#3",
        "Tot",
        "IU Del",
        "CDW#1",
        "CDW#2",
        "Com Pub",
        "Adv Partic",
        "Eva",
        "Status",
    ]
]
td = td.rename(
    columns={
        "TDSP": "tdsp",
        "Ref #": "rid",
        "FC ID #": "tdfcid",
        "CSLT": "cslt_session",
        "CSNT": "csnt_session",
        "LD": "ld",
        "IU": "iu",
        "CW": "cdw",
        "Com": "comm",
        "Adv": "adv",
        "LD ADMSP": "ld_admsp",
        "LD SJAC": "ld_sjac",
        "IS#1": "iu_s1",
        "IS#2": "iu_s2",
        "IS#3": "iu_s3",
        "Tot": "iu_total",
        "IU Del": "iu_del",
        "CDW#1": "cdw_s1",
        "CDW#2": "cdw_s2",
        "Com Pub": "post_date",
        "Adv Partic": "adv_att",
        "Eva": "eva_session",
        "Status": "status",
    }
)

# There are two beneficiaries with no rid. I will use a placeholder value for them, just to help me merge them to the consolidated dataset.
td.loc[0, "rid"] = "M0001"
td.loc[3, "rid"] = "M0002"

In [6]:
cds_td = pd.merge(cds, td, on="rid", how="left")

In [7]:
cds_td.to_csv(
    r"notebooks/consolidate_datasets/dec24/consolidated_dataset_td.csv", index=False
)

In [ ]:
# cds = pd.read_csv('consolidated_dataset10.csv')

In [12]:
# TRW Dataset
trw_file = path_manager.get_data_file(Category.PS, PSFile.TRW)
trw = get_df(trw_file.path, trw_file.sheet, config_file)
trw = trw.drop(columns=["fac1", "fac2", "note", "firstname", "lastname"], axis=1)
# Correct a beneficiary rid
trw.loc[28, "rid"] = "R0358"
# Decide what duplicated rows to drop
trw = trw.drop([60, 29], axis=0)
trw = trw.rename(columns={"fcid": "trwfcid"})

In [11]:
# trw

In [14]:
cds_trw = pd.merge(cds_td, trw, on="rid", how="left")

In [ ]:
cds_trw.to_csv(
    r"notebooks/consolidate_datasets/dec24/consolidated_dataset_trw.csv", index=False
)

In [ ]:
# cds = pd.read_csv('consolidated_dataset11.csv')

In [17]:
pei_file = path_manager.get_data_file(Category.PS, PSFile.PEI)
pei = get_df(pei_file.path, pei_file.sheet, config_file)
pei = pei.drop(columns=["note", "firstname", "lastname"], axis=1)
pei = pei.rename(columns={"fcid": "peifcid"})

In [18]:
cds_pei = pd.merge(cds_trw, pei, on="rid", how="left")

In [19]:
cds_pei.to_csv(
    r"notebooks/consolidate_datasets/dec24/consolidated_dataset_pei.csv", index=False
)

In [ ]:
# cds = pd.read_csv('consolidated_dataset12.csv')

In [23]:
aww_file = path_manager.get_data_file(Category.PS, PSFile.AWW)
aww = get_df(aww_file.path, aww_file.sheet, config_file)
aww = aww.drop(
    columns=["note", "aws_ci", "fcid", "sex", "age", "firstname", "lastname"], axis=1
)
# aww = aww.rename(columns={'rid': 'awwrid'})

In [ ]:
cds_aww = pd.merge(cds_pei, aww, on="rid", how="left")

In [25]:
cds_aww.to_csv(
    r"notebooks/consolidate_datasets/dec24/consolidated_dataset_complete.csv",
    index=False,
)

In [26]:
# cds = pd.read_csv('consolidated_dataset13.csv')
cds_aww.to_excel(
    r"notebooks/consolidate_datasets/dec24/consolidated_dataset_complete.xlsx",
    index=False,
)